In [ ]:
from glob import glob
from os.path import basename

logger = sc._jvm.org.apache.log4j
logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )
print("done")

In [ ]:
# this needs to be in sync with file://retrosheet-spark.ipynb

DIST_PARQUET_DIR = "dist/parquet/*"


for recTypePath in glob(DIST_PARQUET_DIR):
    
    recType = basename(recTypePath)
    df = sqlContext.read.parquet(recTypePath)
    df.registerTempTable(recType)
    print(recType)
    sqlContext.table(recType).printSchema()



In [ ]:
sql = """

    select player_id, firstName, lastName, 
            min(year) as minYear, 
            max(year) as maxYear, 
            count(distinct team) as numTeams
    from roster
    group by player_id, firstName, lastName
    order by numTeams desc
    
"""


sqlContext.sql(sql).show()


In [ ]:

sql = """

select p.*
from play p join (
        select s.player_id, s.game_id 
        from play s 
            join play d on (
                s.player_id = d.player_id 
                and s.game_id = d.game_id
                and s.description like "S%"
                and d.description like "D%"
            )

            join play t on (
                s.player_id = t.player_id 
                and s.game_id = t.game_id
                and t.description like "T%"
            )
    ) sdt on (
        p.player_id = sdt.player_id
        and p.game_id = sdt.game_id
    )
order by p.game_id desc, seq asc

"""

sqlContext.sql(sql).show(100)


In [ ]:

sql = """

select min(gameDate), max(gameDate)
from play 

"""

sqlContext.sql(sql).show()
